In [1]:
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from IPython.display import clear_output, display
from selenium.webdriver.common.by import By
from selenium import webdriver
from time import sleep
from tqdm import tqdm
import pandas as pd
import re

In [2]:
df = pd.read_csv("links.csv")
df

,youtube_link,caption,download_link,success,downloaded
0,https://youtu.be/EYr7tJCf8C4,1__تئوری_آشوب_به_زبان_ساده_جلسه_۱_چارچوب_فیزیک...,https://srv9.dlmate.cc/dl?hash=IkrzfscmE6ls74L...,True,NaN
1,https://youtu.be/5_rSxy7uk-0,2__تئوری_آشوب_به_زبان_ساده_جلسه_۱_چارچوب_فیزیک...,https://srv5.dlmate.cc/dl?hash=8sq97Yz5bMWu%2F...,True,NaN
2,https://youtu.be/rn_WmM45cGc,3__تئوری_آشوب_به_زبان_ساده_جلسه_۲_اجزا_و_رابطه...,https://srv3.dlmate.cc/dl?hash=8Oc0AZqgjnscr1J...,True,NaN
3,https://youtu.be/xH1BOnD0tOE,4__تئوری_آشوب_به_زبان_ساده_جلسه_۳_بودن_و_شدن_ق...,https://srv5.dlmate.cc/dl?hash=yWPwQQrNqFBQfL0...,True,NaN
4,https://youtu.be/tErALPjnCIE,5__تئوری_آشوب_به_زبان_ساده_جلسه_۳_بودن_و_شدن_ق...,https://srv.dlmate.cc/dl?hash=iYe0JhDq8CRMyfNh...,True,NaN
...,...,...,...,...,...
111,https://youtu.be/UHmsyHnKYFg,112__جبر_خطی_جلسه_۹_بردارها_و_مقادیر_ویژه_یک_م...,https://srv6.dlmate.cc/dl?hash=Tz4h%2BlGY52Pm2...,True,NaN
112,https://youtu.be/c3GSThHHcpw,113__جبر_خطی_جلسه_۹_بردارها_و_مقادیر_ویژه_یک_م...,https://srv6.dlmate.cc/dl?hash=8H%2BwIgRmRWhRU...,True,NaN
113,https://youtu.be/sIWtjDZv3aA,114__جبر_خطی_جلسه_۱۰_تابع_اکسپونانسیل_ماتریس_e...,https://srv9.dlmate.cc/dl?hash=CS%2FUNJYC9l%2F...,True,NaN
114,https://youtu.be/owm5Vf1TM7k,115__جبر_خطی_جلسه_۱۰_تابع_اکسپونانسیل_ماتریس_e...,https://srv6.dlmate.cc/dl?hash=uGMYxuI59w%2BYQ...,True,NaN


In [3]:
def sanitize_filename(caption):
    caption = re.sub(r'Duration:\s*\d{1,2}:\d{2}\s*minutes$', '', caption, flags=re.MULTILINE)
    filename = re.sub(r'[^\w\s-]', '', caption)
    filename = re.sub(r'[\s\n]+', '_', filename)
    filename = filename.strip('_')
    return filename[:240]

In [4]:
def save_download_video(youtube_link):
    try:
        url = 'https://en.y2mate.is/v96/'

        chrome_options = Options()
        chrome_options.add_argument('--headless')  # فعال کردن حالت headless
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')

        driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)

        # driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

        driver.get(url)

        driver.maximize_window()

        a = driver.find_elements(By.ID, "txtUrl")[0].send_keys(youtube_link)

        b = driver.find_elements(By.ID, "btnSubmit")[0].click()

        sleep(3)

        trs = driver.find_elements(By.TAG_NAME, 'tr')

        for i in trs:
            if "360p" in i.text or "720p" in i.text:
                tr = i
                break

        tr.find_element(By.TAG_NAME, 'button').click()

        sleep(3)

        count = 0

        while True:

            count += 1

            if count > 20:
                break

            try:
                download_btn = driver.find_element(By.ID, "btnOk")
                break

            except:
                sleep(3)
                continue

        x = download_btn.find_element(By.TAG_NAME, "button").find_element(By.TAG_NAME, "a")

        sleep(3)

        download_link = x.get_attribute("href")

        caption = driver.find_element(By.TAG_NAME, "b").text

        caption = sanitize_filename(caption)

        driver.quit()

        return download_link, caption
    
    except Exception as e:
        print(e)
        return None, None

In [5]:
for index, row in tqdm(df.iterrows()):

    clear_output(wait=True)
    display(f"INDEX : {index}")

    if pd.notna(row['download_link']):
        continue

    download_link, caption = save_download_video(row['youtube_link'])

    caption = f"{index + 1}__{caption}"

    df.at[index, 'download_link'] = download_link
    df.at[index, 'caption'] = caption

    if download_link and caption:
        df.at[index, 'success'] = True

    df.to_csv("links.csv", index=False)

'INDEX : 115'

116it [02:01,  1.04s/it]
